# Modeling

Here we'll build a computer vision model to detect if a lung disease is present or not.

In [4]:
import os
import pandas as pd

## Data Loading

Using the final results from the data_exploration_and_processing notebook, we'll load our data and labels.

In [3]:
def get_image_files(index_lst: list):
    image_files = []

    for i in index_lst:
        image_dir = f"D:\BigData\images_00{i}\images"
        
        image_files_for_one_folder = os.listdir(image_dir)
        
        image_files += image_files_for_one_folder
        
    return image_files
    
image_files = get_image_files([3, 8])
len(image_files)

20000

In [7]:
df = pd.read_csv("../data/lung_disease_labels.csv")
df.head(3)

,image_index,follow-up_#,patient_age,patient_gender,view_position,finding_labels
0,00000001_000.png,0,58,0,0,1
1,00000001_001.png,1,58,0,0,1
2,00000001_002.png,2,58,0,0,1


In [8]:
subset_df = df[df["image_index"].isin(image_files)]
subset_df.shape

(19998, 6)

Missing two images, likely just from the faulty age data we removed. This is fine, let's move on.

## Data Preprocessing

In [ ]:
import cv2
import numpy as np
from PIL import Image

def opencv_preprocessing(image_path, final_size=(224, 224)):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    image = cv2.equalizeHist(image)
    
    image = cv2.resize(image, final_size)
    
    # Optional: Clip pixel values to be in the valid range (0-255)
    image = np.clip(image, 0, 255).astype('uint8')
    
    # Convert from OpenCV image (NumPy array) to PIL image for Torch transformations
    pil_image = Image.fromarray(image)
    
    return pil_image


In [ ]:
import torchvision.transforms as transforms

def torch_preprocessing():
    return transforms.Compose([
        transforms.RandomHorizontalFlip(),       # Apply random horizontal flip
        transforms.RandomRotation(10),           # Apply random rotation of ±10 degrees
        transforms.ToTensor(),                   # Convert image to tensor
        transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize the pixel values
    ])

In [9]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load a pretrained ResNet model for binary or multi-label classification
model = models.resnet50(pretrained=True)

c:\Users\RaviB\anaconda3\envs\healthscan\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\RaviB\anaconda3\envs\healthscan\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\RaviB/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:09<00:00, 10.8MB/s]
